In [6]:
import pandas as pd
import requests
import datetime
from dateutil import parser
from tqdm.auto import tqdm
import json
import re

# -----------------------------------
# 1. Read the CSV data
# -----------------------------------
df_bills = pd.read_csv("crec2023.csv")

# Remove rows where speaker starts with any of The CLERK, The SPEAKER, The PRESIDENT, The PRESIDING OFFICER, The Acting CHAIR, The CHAIR, The ACTING PRESIDENT case-insensitive
df_bills = df_bills[~df_bills["speaker"].str.lower().str.startswith(("the clerk", "the speaker", "the president", "the presiding officer", "the acting chair", "the chair", "the acting president"), na=False)]

# Filter out rows where speech is less than 250 characters, to filter out procedural speehces
df_bills = df_bills[df_bills['speech'].str.len() >= 250]
# -----------------------------------
# 2. Identify potential bills
#    Simple criterion: title contains the word 'ACT'
# -----------------------------------

# Drop or fill missing values in the 'title' column
df_bills = df_bills.dropna(subset=["title"])

# Filter rows where 'title' contains ' ACT'
df_bills = df_bills[df_bills["title"].str.contains(" ACT", na=False, case=False)]  # Ignore case if needed

def preprocess_text(text):
    # Replace all consecutive whitespaces with a single whitespace
    text = re.sub(r'\s+', ' ', text)
    # Replace Madam Speaker, Mr. President, Madam President with Mr. Speaker
    text = re.sub(r'Mr\. President', 'Mr. Speaker', text)
    text = re.sub(r'Mr\. Clerk', 'Mr. Speaker', text)
    text = re.sub(r'Mr\. Chair', 'Mr. Speaker', text)
    text = re.sub(r'Mr\. Chairman', 'Mr. Speaker', text)
    text = re.sub(r'Mr\. Speakerman', 'Mr. Speaker', text)
    text = re.sub(r'Madam President', 'Mr. Speaker', text)
    text = re.sub(r'Madam Speaker', 'Mr. Speaker', text)
    text = re.sub(r'Madam Clerk', 'Mr. Speaker', text)
    text = re.sub(r'Madam Chair', 'Mr. Speaker', text)
    text = re.sub(r'Madam Chairman', 'Mr. Speaker', text)
    text = re.sub(r'Madam Chairwoman', 'Mr. Speaker', text)

    # "Mr. Speaker, " 
    text = re.sub(r'^Mr\. Speaker, ', '', text)
    # Strike starting sentence that starts with "I yield"
    text = re.sub(r'^I yield.*?\. *', '', text)
    # Remove second "Mr. Speaker, " that sometimes follows
    text = re.sub(r'^Mr\. Speaker, ', '', text)

    return text

df_bills['speech'] = df_bills.apply(lambda x: preprocess_text(x['speech']), axis=1)


# # Write df_bills to csv
df_bills.to_csv("df_bills.csv", index=False)

df_bills.head()


,speech_id,speech,chamber,is_extension,date,speaker,speaker_bioguide,vol,num,congress_num,pages,doc_title,title
58,CREC-2023-01-09-pt1-PgE3-4-2,I rise today to introduce the District of Colu...,H,True,1/9/2023,Ms. NORTON,N000147,169,7,118,E3-E4,"INTRODUCTION OF THE WASHINGTON, D.C. ADMISSION...","INTRODUCTION OF THE WASHINGTON, D.C. ADMISSION..."
198,CREC-2023-01-09-pt1-PgH76-12,the process for considering our first bill of ...,H,False,1/9/2023,Mr. SMITH of Nebraska,S001172,169,7,118,H76-H94,FAMILY AND SMALL BUSINESS TAXPAYER PROTECTION ACT,FAMILY AND SMALL BUSINESS TAXPAYER PROTECTION ACT
199,CREC-2023-01-09-pt1-PgH76-13,I rise in strong opposition to H.R. 23. My fri...,H,False,1/9/2023,Mr. NEAL,N000015,169,7,118,H76-H94,FAMILY AND SMALL BUSINESS TAXPAYER PROTECTION ACT,FAMILY AND SMALL BUSINESS TAXPAYER PROTECTION ACT
200,CREC-2023-01-09-pt1-PgH76-15,he points out in the opening paragraph of a ta...,H,False,1/9/2023,Mr. NEAL,N000015,169,7,118,H76-H94,FAMILY AND SMALL BUSINESS TAXPAYER PROTECTION ACT,FAMILY AND SMALL BUSINESS TAXPAYER PROTECTION ACT
201,CREC-2023-01-09-pt1-PgH76-16,I include in the Record an excerpt from a 2021...,H,False,1/9/2023,Mr. SMITH of Nebraska,S001172,169,7,118,H76-H94,FAMILY AND SMALL BUSINESS TAXPAYER PROTECTION ACT,FAMILY AND SMALL BUSINESS TAXPAYER PROTECTION ACT


In [2]:
API_KEY = "api_key"
BASE_URL = "https://api.congress.gov/v3"
CONGRESS_NUMBER = 118

In [12]:
url = f"{BASE_URL}/bill/{CONGRESS_NUMBER}/{'hr'}/{3691}/subjects"
params = {"api_key": API_KEY, "format": "json"}
resp = requests.get(url, params=params)
data = resp.json()
data

{'pagination': {'count': 7},
 'request': {'billNumber': '3691',
  'billType': 'hr',
  'billUrl': 'https://api.congress.gov/v3/bill/118/hr/3691?format=json',
  'congress': '118',
  'contentType': 'application/json',
  'format': 'json'},
 'subjects': {'legislativeSubjects': [{'name': 'Aviation and airports',
    'updateDate': '2024-01-08T20:30:29Z'},
   {'name': 'Congressional oversight', 'updateDate': '2024-01-08T20:30:29Z'},
   {'name': 'Emergency medical services and trauma care',
    'updateDate': '2024-01-08T20:30:29Z'},
   {'name': 'Government studies and investigations',
    'updateDate': '2024-01-08T20:30:29Z'},
   {'name': 'Health care costs and insurance',
    'updateDate': '2024-01-08T20:30:29Z'},
   {'name': 'Medicare', 'updateDate': '2024-01-08T20:30:29Z'}],
  'policyArea': {'name': 'Health', 'updateDate': '2023-06-21T15:46:05Z'}}}

In [15]:
data.get("subjects").get("policyArea").get("name")

'Health'

In [3]:
def get_bill_subjects_and_policy_area(api_key, congress, bill_type, bill_number):
    """
    Returns:
        {
           "policy_area": {
               "name": str,
               "updateDate": str or None
           },
           "legislative_subjects": [
               {
                 "name": str,
                 "updateDate": str or None
               },
               ...
           ]
        }
    or an empty dict if no data is found or request fails.
    """
    url = f"{BASE_URL}/bill/{congress}/{bill_type.lower()}/{bill_number}/subjects"
    params = {"api_key": api_key, "format": "json"}
    resp = requests.get(url, params=params)
    if resp.status_code != 200:
        return {}
    data = resp.json()
    
    subjects_data = data.get("subjects", {})
    
    # Policy area
    policy_area = subjects_data.get("policyArea", {})
    
    # Legislative subjects (list)
    legislative_subjects = subjects_data.get("legislativeSubjects", [])
    
    return {
        "policy_area": {
            "name": policy_area.get("name", ""),
            "updateDate": policy_area.get("updateDate")
        },
        "legislative_subjects": [
            {
                "name": s.get("name", ""),
                "updateDate": s.get("updateDate")
            }
            for s in legislative_subjects
        ]
    }

def get_bill_summaries_all(api_key, congress, bill_type, bill_number):
    """
    Returns a list of all summaries, each item:
        {
          "actionDate": str (YYYY-MM-DD),
          "actionDesc": str,
          "text": str,
          "updateDate": str,
          "versionCode": str
        }
    """
    url = f"{BASE_URL}/bill/{congress}/{bill_type.lower()}/{bill_number}/summaries"
    params = {"api_key": api_key, "format": "json"}
    resp = requests.get(url, params=params)
    if resp.status_code != 200:
        return []
    data = resp.json()
    return data.get("summaries", [])

def get_bill_committees_all(api_key, congress, bill_type, bill_number):
    """
    Returns a list of committees. Each committee looks like:
        {
            "name": str,
            "chamber": str,
            "type": str,
            "activities": [
               {"date": "2023-05-29T18:00:30Z", "name": "Referred To"}, ...
            ],
            "subcommittees": [
               {
                  "name": "...",
                  "activities": [...],
                  ...
               },
               ...
            ]
        }
    """
    url = f"{BASE_URL}/bill/{congress}/{bill_type.lower()}/{bill_number}/committees"
    params = {"api_key": api_key, "format": "json"}
    resp = requests.get(url, params=params)
    if resp.status_code != 200:
        return []
    data = resp.json()
    return data.get("committees", [])


In [4]:
def fetch_and_cache_bill_data(api_key, congress, bill_type, bill_number):
    """
    Fetches all the relevant data for a single bill
    (subjects, policy area, all summaries, committees, etc.)
    Returns a dict with keys:
        "bill_type", "bill_number", "subjects_policy", "summaries", "committees"
    """
    result = {}
    # subjects & policy area
    result["subjects_policy"] = get_bill_subjects_and_policy_area(api_key, congress, bill_type, bill_number)
    # summaries
    result["summaries"] = get_bill_summaries_all(api_key, congress, bill_type, bill_number)
    # committees
    result["committees"] = get_bill_committees_all(api_key, congress, bill_type, bill_number)
    return result


In [5]:
all_bills_dict = {}

In [10]:
def build_bill_data_dict(api_key, all_bills, congress=118, limit_per_page=250, offset=0):
    """
    Returns a dictionary: 
      { 
          <lowercase bill title> : {
               "title": str,
               "bill_type": str,
               "bill_number": str,
               "url": str,
               "updateDate": str,
               # plus the deeper fetched data:
               "subjects_policy": {...},
               "summaries": [...],
               "committees": [...]
          },
          ...
      }
    """
    report = 1000
    while True:
        params = {
            "api_key": api_key,
            "format": "json",
            "offset": offset,
            "limit": limit_per_page
        }
        url = f"{BASE_URL}/bill/{congress}"
        resp = requests.get(url, params=params)
        resp.raise_for_status()
        data = resp.json()
        
        bills_page = data.get("bills", [])
        if not bills_page:
            break
        
        # For each bill in this page, store minimal metadata
        for b in bills_page:
            offset += 1
            b_title_lower = b.get("title", "").lower()
            if b_title_lower not in all_bills:
                all_bills[b_title_lower] = {
                    "title": b.get("title", "").strip("."),
                    "bill_type": b.get("type", ""),
                    "bill_number": b.get("number", ""),
                    "url": b.get("url", ""),
                    "updateDate": b.get("updateDate", "")
                }

        if offset >= report:
            print(offset)
            report += 1000


build_bill_data_dict(API_KEY, all_bills_dict, CONGRESS_NUMBER, offset=0)

with open("all_bills_dict.json", "w") as f:
    json.dump(all_bills_dict, f, indent=2)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [7]:
with open("all_bills_dict.json", "r") as f:
    all_bills_dict = json.load(f)

In [13]:
# Now for each stored bill, fetch deeper data (subjects, committees, etc.)
for title_lower, info in tqdm(all_bills_dict.items()):
    # if we already have the deeper data, skip
    bt = info["bill_type"]
    bn = info["bill_number"]
    # fetch the deeper data
    fetched = fetch_and_cache_bill_data(API_KEY, CONGRESS_NUMBER, bt, bn)
    info.update(fetched)

with open("all_bills_dict.json", "w") as f:
    json.dump(all_bills_dict, f, indent=2)

  0%|          | 15/16020 [00:23<7:05:28,  1.60s/it]


KeyboardInterrupt: 

In [13]:
with open("all_bills_dict.json", "r") as f:
    all_bills_dict = json.load(f)

In [8]:
def standardize_bill_title(title):
    """
    Standardize a bill title by:
    1. Converting to lowercase
    2. Combining multiple spaces into single space
    3. Removing specific prefixes and suffixes
    4. Removing specific procedural phrases
    
    Args:
        title (str): Original bill title
        
    Returns:
        str: Standardized bill title, or None if input is None/empty
    """
    if not title:
        return None
        
    # Convert to lowercase
    title = title.lower()
    
    # Combine multiple spaces into one
    title = ' '.join(title.split())
    
    # Skip titles that begin with "authorizing the clerk"
    if title.startswith("authorizing the clerk"):
        return None
        
    # Remove various prefixes up to the next comma
    prefixes_with_comma = [
        "providing for consideration of",
        "providing for further consideration of",
        "report on"
    ]
    
    for prefix in prefixes_with_comma:
        if title.startswith(prefix):
            comma_pos = title.find(", ")
            if comma_pos != -1:
                title = title[comma_pos + 2:]  # +2 to skip the comma and space
                break
    
    # Remove simple prefixes
    simple_prefixes = [
        "introduction of the ",
        "reintroduction of the "
    ]
    
    for prefix in simple_prefixes:
        if title.startswith(prefix):
            title = title[len(prefix):]
            
    # Remove trailing period or "--"
    if title.endswith("."):
        title = title[:-1]
    if title.endswith("--"):
        title = title[:-2]
        
    # Trim any resulting whitespace
    title = title.strip()
    
    return title if title else None

def get_bill_data(title, all_bills_dict):
    """
    Get the bill data (policy area, legislative subjects, summary, committees).
    Date checks removed - will return most recent data regardless of speech date.
    
    Args:
        title (str): Title of the bill
        all_bills_dict (dict): Dictionary containing bill data
        
    Returns:
        dict: Dictionary containing relevant bill data, or None if not found
    """
    # Standardize the title
    title_std = standardize_bill_title(title)
    if not title_std:
        return None
    
    # Try to find the bill in the dictionary
    if title_std not in all_bills_dict:
        return None
    
    bill_data = all_bills_dict[title_std]
    
    result = {
        'bill_type': bill_data.get('bill_type') or None,
        'bill_number': bill_data.get('bill_number') or None,
        'policy_area': None,
        'policy_area_update': None,
        'legislative_subjects': [],
        'latest_summary': None,
        'latest_summary_date': None,
        'committees': []
    }
    
    # Get policy area
    policy_area_data = bill_data.get('subjects_policy', {}).get('policy_area', {})
    if policy_area_data:
        result['policy_area'] = policy_area_data.get('name') or None
        if policy_area_data.get('updateDate'):
            result['policy_area_update'] = parser.parse(policy_area_data['updateDate']).date().isoformat()
    
    # Get legislative subjects
    leg_subjects = bill_data.get('subjects_policy', {}).get('legislative_subjects', [])
    for subject in leg_subjects:
        subject_data = {
            'name': subject.get('name') or None,
            'updateDate': parser.parse(subject['updateDate']).date().isoformat() if subject.get('updateDate') else None
        }
        result['legislative_subjects'].append(subject_data)
    
    # Get most recent summary
    summaries = bill_data.get('summaries', [])
    if summaries:
        # Get the most recent summary
        latest_summary = max(summaries, key=lambda x: parser.parse(x['actionDate']).date() if x.get('actionDate') else parser.parse('1900-01-01').date())
        result['latest_summary'] = latest_summary.get('text')
        if latest_summary.get('actionDate'):
            result['latest_summary_date'] = parser.parse(latest_summary['actionDate']).date().isoformat()
    
    # Get committees with their subcommittees
    committees = bill_data.get('committees', [])
    
    for committee in committees:
        committee_data = {
            'name': committee.get('name') or None,
            'chamber': committee.get('chamber') or None,
            'type': committee.get('type') or None,
            'latest_activity': None,
            'subcommittees': []
        }
        
        # Get latest committee activity
        activities = committee.get('activities', [])
        if activities:
            latest_activity = max(activities, key=lambda x: parser.parse(x['date']).date() if x.get('date') else parser.parse('1900-01-01').date())
            if latest_activity.get('date'):
                committee_data['latest_activity'] = parser.parse(latest_activity['date']).date().isoformat()
        
        # Get subcommittees
        for subcommittee in committee.get('subcommittees', []):
            sub_activities = subcommittee.get('activities', [])
            if sub_activities:
                latest_sub_activity = max(sub_activities, key=lambda x: parser.parse(x['date']).date() if x.get('date') else parser.parse('1900-01-01').date())
                if latest_sub_activity.get('date'):
                    subcommittee_data = {
                        'name': subcommittee.get('name') or None,
                        'latest_activity': parser.parse(latest_sub_activity['date']).date().isoformat()
                    }
                    committee_data['subcommittees'].append(subcommittee_data)
        
        result['committees'].append(committee_data)
    
    return result


# Before processing df_bills, standardize all titles in all_bills_dict
print("Standardizing bill titles in dictionary...")
standardized_bills_dict = {}
for title, data in all_bills_dict.items():
    std_title = standardize_bill_title(title)
    if std_title:
        standardized_bills_dict[std_title] = data
all_bills_dict = standardized_bills_dict

# Apply the function to each row in df_bills
print("Adding bill data columns...")
bill_data_list = []
for _, row in tqdm(df_bills.iterrows(), total=len(df_bills)):
    bill_data = get_bill_data(row['title'], all_bills_dict)
    bill_data_list.append(bill_data if bill_data else {})

# Add new columns to df_bills
df_bills['standardized_title'] = df_bills['title'].apply(standardize_bill_title)
df_bills['bill_type'] = [data.get('bill_type') for data in bill_data_list]
df_bills['bill_number'] = [data.get('bill_number') for data in bill_data_list]
df_bills['policy_area'] = [data.get('policy_area') for data in bill_data_list]
df_bills['legislative_subjects'] = [data.get('legislative_subjects', []) for data in bill_data_list]
df_bills['latest_summary'] = [data.get('latest_summary') for data in bill_data_list]
df_bills['committees'] = [data.get('committees', []) for data in bill_data_list]

print("Done!")

Standardizing bill titles in dictionary...
Adding bill data columns...


100%|██████████| 5555/5555 [00:03<00:00, 1409.84it/s]

Done!


In [9]:
# write df_bills to csv
df_bills.to_csv("df_bills.csv", index=False)

In [5]:
# list speakers from df_bills, sorted alphabetically
sorted(df_bills['title'].unique())

['ACCREDITED INVESTOR DEFINITION REVIEW ACT',
 'ACCURATELY COUNTING RISK ELIMINATION SOLUTIONS ACT',
 'ADVANCED, LOCAL EMERGENCY RESPONSE TELECOMMUNICATIONS PARITY ACT',
 'AFFIRMATIVE ACTION',
 'AFFIRMATIVE ACTION DECISION',
 'AFFORDABLE CARE ACT',
 'AGRICULTURE, RURAL DEVELOPMENT, FOOD AND DRUG ADMINISTRATION, AND  RELATED AGENCIES APPROPRIATIONS ACT, 2024',
 'AIRPORT AND AIRWAY EXTENSION ACT OF 2023, PART II',
 'ALL-AMERICAN FLAG ACT',
 'AMENDING THE FEDERAL ELECTION CAMPAIGN ACT OF 1971 TO EXTEND THE  ADMINISTRATIVE FINE PROGRAM FOR CERTAIN REPORTING VIOLATIONS',
 'AMENDING THE INVESTMENT ADVISERS ACT OF 1940 TO CODIFY CERTAIN  SECURITIES AND EXCHANGE COMMISSION NO-ACTION LETTERS THAT EXCLUDE',
 'AMERICAN BATTLEFIELD PROTECTION PROGRAM ENHANCEMENT ACT OF 2023',
 'AMERICANS ARE DEMANDING ACTION',
 "APPLICABILITY OF THE AIRLINE DEREGULATION ACT TO STATE WORKERS'  COMPENSATION PROGRAMS",
 'ARMENIAN PROTECTION ACT OF 2023',
 'AUTHORIZING THE CLERK TO MAKE CORRECTIONS IN ENGROSSMENT OF H